In [8]:
%run ../Python_files/util.py

In [ ]:
##### read in raw data

import openpyxl

data_folder = '/home/jzh/Dropbox/Research/\
Data-driven_estimation_inverse_optimization/INRIX/Raw_data/'

# load filtered INRIX attribute table raw data
wb_INRIX = openpyxl.load_workbook(data_folder + 'filtered_INRIX_attribute_table_ext.xlsx')

# load filtered capacity attribute table raw data
wb_capac = openpyxl.load_workbook(data_folder + 'capacity_attribute_table.xlsx')

In [3]:
# get sheet name from workbook
sheet_INRIX_name = wb_INRIX.sheetnames[0].encode('utf-8')
sheet_capac_name = wb_capac.sheetnames[0].encode('utf-8')

In [4]:
# get sheet of filtered INRIX attribute table raw data
sheet_INRIX = wb_INRIX.get_sheet_by_name(sheet_INRIX_name)

# get sheet of capacity attribute table raw data
sheet_capac = wb_capac.get_sheet_by_name(sheet_capac_name)

In [47]:
# road_invent, length, route_num, AB_AM_capac, AB_MD_capac, AB_PM_capac, AB_NT_capac, 
# AB_AM_lane, AB_MD_lane, AB_PM_lane, AB_NT_lane

road_invent = str(sheet_capac.cell(row=2, column=26).value)
length = sheet_capac.cell(row=2, column=2).value
route_num = str(sheet_capac.cell(row=2, column=8).value)
AB_AM_capac = float(sheet_capac.cell(row=2, column=18).value)
AB_MD_capac = float(sheet_capac.cell(row=2, column=20).value)
AB_PM_capac = float(sheet_capac.cell(row=2, column=22).value)
AB_NT_capac = float(sheet_capac.cell(row=2, column=24).value)
AB_AM_lane = int(sheet_capac.cell(row=2, column=10).value)
AB_MD_lane = int(sheet_capac.cell(row=2, column=12).value)
AB_PM_lane = int(sheet_capac.cell(row=2, column=14).value)
AB_NT_lane = int(sheet_capac.cell(row=2, column=16).value)

RoadSegCapacLane_dict = {}

RoadSegCapacLane_dict[road_invent] = \
                RoadSegCapacLane(road_invent, length, route_num, AB_AM_capac, \
                AB_MD_capac, AB_PM_capac, AB_NT_capac, AB_AM_lane, \
                AB_MD_lane, AB_PM_lane, AB_NT_lane)

In [48]:
RoadSegCapacLane_dict

{'19327900': <__main__.RoadSegCapacLane at 0x7f93000cff90>}

In [39]:
length, route_num, AB_AM_capac, AB_MD_capac, AB_PM_capac, AB_NT_capac, AB_AM_lane, AB_MD_lane, AB_PM_lane, AB_NT_lane

(0.082878, '80', 2062.5, 3918.75, 2062.5, 5775.0, 1, 1, 1, 3)

In [1]:
# load lookup table raw data
wb_lookup = openpyxl.load_workbook(data_folder + 'roadinv_id_to_tmc_lookup.xlsx') 

# get sheet name from workbook
sheet_INRIX_name = wb_INRIX.sheetnames[0].encode('utf-8')
sheet_capac_name = wb_capac.sheetnames[0].encode('utf-8')
sheet_lookup_name = wb_lookup.sheetnames[0].encode('utf-8')

# get sheet of filtered INRIX attribute table raw data
sheet_INRIX = wb_INRIX.get_sheet_by_name(sheet_INRIX_name)

# get sheet of filtered capacity attribute table raw data
sheet_capac = wb_capac.get_sheet_by_name(sheet_capac_name)

# get sheet of lookup table raw data
sheet_lookup = wb_lookup.get_sheet_by_name(sheet_lookup_name)


##### extract attributes of interest from INRIX sheet

tmc_list = []
road_num_list = []
shape_length_list = []

for i in xrange(2, 1 + sheet_INRIX.max_row):
    tmc_list.append(sheet_INRIX.cell(row=i, column=2).value.encode('utf-8'))
    road_num_list.append(sheet_INRIX.cell(row=i, column=3).value.encode('utf-8'))
    shape_length_list.append(sheet_INRIX.cell(row=i, column=13).value)

assert(len(tmc_list) == len(road_num_list) and \
       len(road_num_list) == len(shape_length_list))    


##### extract attributes of interest from capacity sheet

road_invent_list = []
length_list = []
route_num_list = []
AB_AM_capac_list = [] 
AB_MD_capac_list = []
AB_PM_capac_list = []
AB_NT_capac_list = []

for i in xrange(2, 1 + sheet_capac.max_row):
    road_invent_list.append(sheet_capac.cell(row=i, column=26).value) 
    length_list.append(sheet_capac.cell(row=i, column=2).value)
    route_num_list.append(sheet_capac.cell(row=i, column=8).value)
    # take the period capacity factor into consideration
    AB_AM_capac_list.append((1.0/2.5)*sheet_capac.cell(row=i, column=18).value)
    AB_MD_capac_list.append((1.0/4.75)*sheet_capac.cell(row=i, column=20).value)
    AB_PM_capac_list.append((1.0/2.5)*sheet_capac.cell(row=i, column=22).value)
    AB_NT_capac_list.append((1.0/7.0)*sheet_capac.cell(row=i, column=24).value)

assert(len(road_invent_list) == len(length_list) and \
               len(length_list) == len(route_num_list) and \
               len(route_num_list) == len(AB_AM_capac_list) and \
               len(AB_AM_capac_list) == len(AB_MD_capac_list) and \
               len(AB_MD_capac_list) == len(AB_PM_capac_list) and \
               len(AB_PM_capac_list) == len(AB_NT_capac_list))    


##### extract attributes of interest from lookup sheet

road_inv_ID_lookup_list = []
tmc_lookup_list = []

for i in xrange(2, 1 + sheet_lookup.max_row):
    road_inv_ID_lookup_list.append(sheet_lookup.cell(row=i, column=1).value) 
    tmc_lookup_list.append(str(sheet_lookup.cell(row=i, column=4).value))

assert(len(road_inv_ID_lookup_list) == len(tmc_lookup_list))    


# instantiation of RoadSegInr class
road_seg_inr = RoadSegInr(tmc_list, road_num_list, shape_length_list)

# instantiation of RoadSegCapac class
road_seg_capac = RoadSegCapac(road_invent_list, length_list, route_num_list, \
                              AB_AM_capac_list, AB_MD_capac_list, \
                              AB_PM_capac_list, AB_NT_capac_list)

# instantiation of LookUp class
look_up = LookUp(road_inv_ID_lookup_list, tmc_lookup_list)

# make a dictionary from look_up
tmc_roadInv_dict = {i:j for (i, j) in zip(look_up.tmc, look_up.road_inv_ID)}

# make dictionaries from road_seg_capac
roadInv_capac_dict_AM = {i:j for (i, j) in zip(road_seg_capac.road_invent, \
                                               road_seg_capac.AB_AM_capac)}
roadInv_capac_dict_MD = {i:j for (i, j) in zip(road_seg_capac.road_invent, \
                                               road_seg_capac.AB_MD_capac)}
roadInv_capac_dict_PM = {i:j for (i, j) in zip(road_seg_capac.road_invent, \
                                               road_seg_capac.AB_PM_capac)}
roadInv_capac_dict_NT = {i:j for (i, j) in zip(road_seg_capac.road_invent, \
                                               road_seg_capac.AB_NT_capac)}

capac_AM = []
capac_MD = []
capac_PM = []
capac_NT = []

for i in range(len(road_seg_inr.tmc)):
    capac_AM.append(roadInv_capac_dict_AM[tmc_roadInv_dict[road_seg_inr.tmc[i]]])
    capac_MD.append(roadInv_capac_dict_MD[tmc_roadInv_dict[road_seg_inr.tmc[i]]])
    capac_PM.append(roadInv_capac_dict_PM[tmc_roadInv_dict[road_seg_inr.tmc[i]]])
    capac_NT.append(roadInv_capac_dict_NT[tmc_roadInv_dict[road_seg_inr.tmc[i]]])

# instantiation of RoadSegInrCapac class
road_seg_inr_capac = RoadSegInrCapac(tmc_list, road_num_list, shape_length_list, \
                                     capac_AM, capac_MD, capac_PM, capac_NT)

zdump(road_seg_inr_capac, '../temp_files/road_seg_inr_capac_ext.pkz')

No dicts found; please check load_dicts...


KeyError: 8120001L